In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import dask.dataframe as dd
from datetime import timedelta
from datetime import datetime

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
person_headers = pd.read_csv('../data/raw/acquia_samples/scripts-headers/person-headers.txt', delimiter='\t')

In [ ]:
person_columns = person_headers.columns

Looks to me like the following tables have been cleaned/analyzed:
* person_identifier
* touch
* matched_segment

currently working on:
* person

In [ ]:

person_ids_gt_one_sorted = dd.read_csv('Z:\\person_ids_gt_one_sorted.csv',names=['person_id'], dtype={'person_id':np.float64})
person_ids_gt_one_sorted = dd.read_csv('Z:\\event_desktop_business_person_ids.tsv',header=0,names=['index','person_id'], dtype={'person_id':np.float64})

In [ ]:
person_ids_gt_one_sorted.compute()

In [ ]:
person_ranking_summ_headers = "person_id, customer_id, customer_site_id, site_name, site_external_id, column_meta_data_id, column_meta_data_name, item_name, frequency, freq_rank, last_modified_date, db_last_modified_date".split(', ')

In [ ]:
person_ranking_summ = dd.read_csv('Z:\\person_ranking_summary.tsv', names=person_ranking_summ_headers, delimiter='\t',
                                  dtype={'column_meta_data_id':np.float64, 'freq_rank':np.float64, 'frequency':np.float64, 'person_id':np.float64})

In [ ]:
person_ranking_summ[person_ranking_summ['person_id'].isin(person_ids_gt_one_sorted['person_id'].compute())].to_csv('Z:\\person_ranking_summary_desktop_business.tsv')

In [ ]:
person_ranking_summ[person_ranking_summ['person_id'].isin(person_ids_gt_one_sorted['person_id'].compute())].to_csv('Z:\\person_ranking_summary_filtered.tsv')

In [ ]:
person_ranking_summ_filtered =  dd.read_csv('Z:\\person_ranking_summary_desktop_business.tsv\*', header=0, names= ['']+person_ranking_summ_headers, delimiter=',',
                                  dtype={'':np.float64, 'column_meta_data_id':np.float64, 'freq_rank':np.float64, 'frequency':np.float64, 'person_id':np.float64})
#prs_value_counts = person_ranking_summ_filtered.item_name.value_counts().compute()
#for x in prs_value_counts.index:
#    print(x, prs_value_counts[x])

In [ ]:
person_ranking_summ_filtered = person_ranking_summ_filtered.drop_duplicates(['person_id', 'item_name'])


In [ ]:
person_ranking_summ_filtered[person_ranking_summ_filtered['item_name'] == 'Landlord/Tenant']['person_id'].compute().to_csv('Z:\\person_id_landlord_tenant.csv')

In [ ]:

#drop duplicates of the person_id, item_id
# 37337, 37341, 37336, 37324, 38561, 39943, 39645
prs_value_counts = person_ranking_summ_filtered[person_ranking_summ_filtered['column_meta_data_id'].isin([37337])].item_name.value_counts().compute()
for x in prs_value_counts.index:
    print(x, prs_value_counts[x])

In [ ]:
%matplotlib inline


prs_value_counts[prs_value_counts.index[:100]].plot(kind='bar', figsize=(20,10))
plt.ylabel('number of people with tag')
plt.title('Tag distribution for people on desktop during business hours', fontsize=24)
plt.savefig('../reports/desktop_business_tag_distribution.png')

## looking at the distribution of terms across people who search in en vs es

In [ ]:
people_en = person_ranking_summ_filtered[person_ranking_summ_filtered['item_name'] == 'en']['person_id'].value_counts().index.compute()
prsf_en = person_ranking_summ_filtered[person_ranking_summ_filtered['person_id'].isin(people_en)]

people_es = person_ranking_summ_filtered[person_ranking_summ_filtered['item_name'] == 'es']['person_id'].value_counts().index.compute()
prsf_es = person_ranking_summ_filtered[person_ranking_summ_filtered['person_id'].isin(people_es)]


In [ ]:
prs_value_counts_en = prsf_en[prsf_en['column_meta_data_id'].isin([37337])].item_name.value_counts().compute()
for x in prs_value_counts_en.index:
    print(x, prs_value_counts_en[x])

In [ ]:
prs_value_counts_es = prsf_es[prsf_es['column_meta_data_id'].isin([37337])].item_name.value_counts().compute()
for x in prs_value_counts_es.index:
    print(x, prs_value_counts_es[x])

In [ ]:
%matplotlib inline


prs_value_counts_en[prs_value_counts_en.index[:100]].plot(kind='bar', figsize=(20,10))
plt.ylabel('number of people with tag')
plt.title('Tag distribution for people with EN tag', fontsize=24)
plt.savefig('../reports/en_tag_distribution.png')

In [ ]:
%matplotlib inline

prs_value_counts_es[prs_value_counts_es.index[:100]].plot(kind='bar', figsize=(20,10))
plt.ylabel('number of people with tag')
plt.title('Tag distribution for people with ES tag', fontsize=24)
plt.savefig('../reports/es_tag_distribution.png')

In [ ]:
## make a column st for every user we have their deduped names, not sorted

prsf_en = prsf_en[prsf_en['column_meta_data_id'].isin([37337, 37341, 37336, 37324, 38561, 39943, 39645])]
prsf_es = prsf_es[prsf_es['column_meta_data_id'].isin([37337, 37341, 37336, 37324, 38561, 39943, 39645])]

prsf_en['combined_items'] = prsf_en['person_id']\
                           .apply(lambda x: "---".join(prsf_en[prsf_en['person_id'] == x]['item_name'].values ), meta=str).compute()

In [ ]:

prsf_es['combined_items'] = prsf_es['person_id']\
                           .apply(lambda x: "---".join(prsf_es[prsf_es['person_id'] == x]['item_name'].values.compute() ), meta=str).compute()

In [ ]:
help(person_ranking_summ_filtered.drop_duplicates())

In [ ]:
pd.__version__

In [ ]:
import dask 
dask.__version__

In [ ]:
person_ranking_summ

In [ ]:
prs_value_counts = person_ranking_summ.item_name.value_counts().compute()
for x in prs_value_counts.index:
    print(x, prs_value_counts[x])

In [ ]:
with open('Z:\\segment.tsv', 'r') as fp:
    for line in fp:
        print(line)

In [ ]:
matched_segment = dd.read_csv('Z:\\matched_segment.tsv', delimiter='\t', names=['event_id', 'segment_id', 'person_id', 'event_date'],
                             dtype={'touch1':np.int64, 'event_id':np.int64, 'touch2':np.int64})

In [ ]:
matched_segment

In [ ]:
len(matched_segment)


In [ ]:
matched_segment.segment_id.value_counts().compute()

In [ ]:
matched_segment.person_id.value_counts().compute()

In [ ]:
'''
touch id 1 - int64   pretty sure event_id?
segment id   - int64  
touch id 2 - int64   pretty sure person id?
session date - datetime pretty sure event date?
'''

In [ ]:
with open('Z:\\matched_segment.tsv', 'r', encoding='utf-8') as f:
    for i in range(2000000):
        l = f.readline()
        if i % 10000 == 0:
            print(l)

In [ ]:
fp_out = open('Z:\\person_ranking_summary_filtered.tsv', 'w', encoding='utf-8')
fp_in = open('Z:\\person_ranking_summary.tsv')
fp_in.readline()
ctr = 0
with open('Z:\\person_ids_gt_one_sorted.csv', 'r') as fp:
    for row in fp:
        in_row = fp_in.readline()
        #if ctr % 50000 == 0:
        #    print(ctr)
        #ctr+=1
        while in_row.startswith(str(row.strip(' \n'))):
            fp_out.write(in_row)
            in_row = fp_in.readline()
            
fp_out.close()
            

In [ ]:
person_identifier = dd.read_csv('Z:\\person_identifier.tsv', delimiter='\t', 
                                names=['person_id', 'customer_id', 'identifier',
                                       'identifier_type', '?', 'active', 'last_modified',
                                       'db_last_modified'])

In [ ]:
len(person_identifier)

In [ ]:
### person columns
person_headers = '''
id	engagement_score	last_touch	first_touch	customer_id	first_time_visitor	subscriber_status	subscription_plan	subscription_start_date	subscription_end_date	last_subscription_plan	articles_remaining	external_score	primary_identiifer	primary_identify_type	unknown	persona	active	last_modified_date	anonymous_visitor	do_not_track	identifiers_json	custom_field_1	custom_field_2	custom_field_3	custom_field_4	custom_field_5	custom_field_6	custom_field_7	custom_field_8	custom_field_9	custom_field_10	custom_field_11	custom_field_12	custom_field_13	custom_field_14	custom_field_15	custom_field_16	custom_field_17	custom_field_18	custom_field_19	custom_field_20	custom_field_21	custom_field_22	custom_field_23	custom_field_24	custom_field_25	custom_field_26	custom_field_27	custom_field_28	custom_field_29	custom_field_30	custom_field_31	custom_field_32	custom_field_33	custom_field_34	custom_field_35	custom_field_36	custom_field_37	custom_field_38	custom_field_39	custom_field_40	custom_field_41	custom_field_42	custom_field_43	custom_field_44	custom_field_45	custom_field_46	custom_field_47	custom_field_48	custom_field_49	custom_field_50	unknown2	unknown3	db_last_modifed_date'''.split('\t')

In [ ]:
person_headers

In [ ]:
with open('Z:\\event.tsv', 'r', encoding='utf-8') as fp:
    print(fp.readline())

In [ ]:
with open('Z:\\event_desktop_business_ppl.tsv', 'r') as fp:
    event_headers = fp.readline().split('\t')

In [ ]:
event_df_pro = dd.read_csv('Z:\\event_desktop_business_ppl.tsv', delimiter='\t', 
                                names=event_headers)

In [ ]:
event_df_cas = dd.read_csv('Z:\\event_mobile_night_ppl.tsv', delimiter='\t', 
                                names=event_headers)

In [ ]:
pd.Series(event_df_pro['person_id'].value_counts().index.compute()).to_csv('Z:\\event_desktop_business_person_ids.tsv')

In [ ]:
pd.Series(event_df_cas['person_id'].value_counts().index.compute()).to_csv('Z:\\event_mobile_night_person_ids.tsv')